[View in Colaboratory](https://colab.research.google.com/github/arpitsharma9/01_machine_learning/blob/master/02_cnn/01_bottleneck_features.ipynb)

# Artificial Intelligence Nanodegree

## Convolutional Neural Networks

---

In your upcoming project, you will download pre-computed bottleneck features.  In this notebook, we'll show you how to calculate VGG-16 bottleneck features on a toy dataset.  Note that unless you have a powerful GPU, computing the bottleneck features takes a significant amount of time.

### 1. Load and Preprocess Sample Images

Before supplying an image to a pre-trained network in Keras, there are some required preprocessing steps.  You will learn more about this in the project; for now, we have implemented this functionality for you in the first code cell of the notebook.  We have imported a very small dataset of 8 images and stored the  preprocessed image input as `img_input`.  Note that the dimensionality of this array is `(8, 224, 224, 3)`.  In this case, each of the 8 images is a 3D tensor, with shape `(224, 224, 3)`.

In [10]:
#@title
from google.colab import drive
drive.mount("/content/drive")


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [11]:
!ls "/content/drive/My Drive"

 Colab
'Colab Notebooks'
'resource%20and%20timesheet%20templates (1).xlsx'
'resource%20and%20timesheet%20templates (1).xlsx.gdoc'
 resource%20and%20timesheet%20templates.xlsx
 resource%20and%20timesheet%20templates.xlsx.gdoc
 retailorganizationalchart.pptx
 retailorganizationalchart.pptx.gslides
 sampleexcelresourcemanagement.xlsx
 sampleexcelresourcemanagement.xlsx.gdoc
 write-data-to-text-file.xls
 write-data-to-text-file.xls.gsheet


In [28]:
!ls "/content/drive/My Drive/Colab/01-dataset/figures/vgg16.png"
!ls "/content/drive/My Drive/Colab/01-dataset/figures/vgg16_transfer.png"
 

'/content/drive/My Drive/Colab/01-dataset/figures/vgg16.png'
'/content/drive/My Drive/Colab/01-dataset/figures/vgg16_transfer.png'


In [0]:
img_path="drive/My Drive/Colab/01-dataset/images/*.jpg"
#img_path="drive/My Drive/Colab/01-dataset/images/sopa.jpg"
#print(img_path)

#img = image.load_img(img_path, target_size=(224, 224))


In [23]:
#from keras.applications.vgg16 import preprocess_input

#Python Imaging Library (abbreviated as PIL) (in newer versions known as Pillow) is a free library for the Python programming language
#that adds support for opening, manipulating, and saving many different image file formats. It is available for Windows, Mac OS X and Linux.
from keras.preprocessing import image
import numpy as np
#The glob module finds all the pathnames matching a specified pattern according to the rules used by the Unix shell, although results are returned in arbitrary order.
import glob



#img_paths = glob.glob("images/*.jpg")
## a object containing list of images path will be created.
img_paths = glob.glob(img_path)
print("path of images",img_paths)

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in img_paths]
    print("list of tensorts", len(list_of_tensors))
    #Stack arrays in sequence vertically (row wise).
    return np.vstack(list_of_tensors)

# calculate the image input. you will learn more about how this works the project!
img_input = preprocess_input(paths_to_tensor(img_paths))

print(img_input.shape)

path of images ['drive/My Drive/Colab/01-dataset/images/sopa.jpg', 'drive/My Drive/Colab/01-dataset/images/Curly-coated_retriever_03896.jpg', 'drive/My Drive/Colab/01-dataset/images/Labrador_retriever_06449.jpg', 'drive/My Drive/Colab/01-dataset/images/Brittany_02625.jpg', 'drive/My Drive/Colab/01-dataset/images/American_water_spaniel_00648.jpg', 'drive/My Drive/Colab/01-dataset/images/Labrador_retriever_06455.jpg', 'drive/My Drive/Colab/01-dataset/images/Welsh_springer_spaniel_08203.jpg', 'drive/My Drive/Colab/01-dataset/images/Labrador_retriever_06457.jpg']
list of tensorts 8
(8, 224, 224, 3)


### 2. Recap How to Import VGG-16

Recall how we import the VGG-16 network (including the final classification layer) that has been pre-trained on ImageNet.

![VGG-16 model]("/content/drive/My Drive/Colab/01-dataset/figures/vgg16.png")

In [29]:
from keras.applications.vgg16 import VGG16
model = VGG16()
model.summary()


553467904/553467096 [==============================] - 15s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_p

For this network, `model.predict` returns a 1000-dimensional probability vector containing the predicted probability that an image returns each of the 1000 ImageNet categories.  The dimensionality of the obtained output from passing `img_input` through the model is `(8, 1000)`.  The first value of `8` merely denotes that 8 images were passed through the network.

In [30]:
model.predict(img_input).shape

(8, 1000)

### 3. Import the VGG-16 Model, with the Final Fully-Connected Layers Removed

When performing transfer learning, we need to remove the final layers of the network, as they are too specific to the ImageNet database.  This is accomplished in the code cell below.

![VGG-16 model for transfer learning](figures/vgg16_transfer.png)

In [31]:
from keras.applications.vgg16 import VGG16
model = VGG16(include_top=False)
model.summary()

58892288/58889256 [==============================] - 2s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool

### 4. Extract Output of Final Max Pooling Layer

Now, the network stored in `model` is a truncated version of the VGG-16 network, where the final three fully-connected layers have been removed.  In this case, `model.predict` returns a 3D array (with dimensions $7\times 7\times 512$) corresponding to the final max pooling layer of VGG-16.  The dimensionality of the obtained output from passing `img_input` through the model is `(8, 7, 7, 512)`.  The first value of `8` merely denotes that 8 images were passed through the network.  

In [32]:
print(model.predict(img_input).shape)

(8, 7, 7, 512)


This is exactly how we calculate the bottleneck features for your project!